<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/42_Inference_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 42: Inference Optimization (Fuzja Conv + BN)

Większość architektur (ResNet, YOLO) składa się z bloków: `Conv -> BN -> ReLU`.
Na produkcji `BN` jest zbędnym narzutem obliczeniowym.

**Matematyka:**
1.  Konwolucja: $y = Wx + b$
2.  Batch Norm: $z = \frac{y - \mu}{\sqrt{\sigma^2 + \epsilon}} \cdot \gamma + \beta$

Możemy to przekształcić w **jedną nową Konwolucję**:
$$ z = W_{new}x + b_{new} $$

Gdzie:
$$ W_{new} = W \cdot \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}} $$
$$ b_{new} = (b - \mu) \cdot \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}} + \beta $$

W tej lekcji napiszemy funkcję, która "połyka" Batchnorma i wypluwa zoptymalizowaną warstwę Conv.

In [1]:
import torch
import torch.nn as nn
import copy
import time

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1. TWORZYMY MODEL "NIEZOPTYMALIZOWANY"
# Conv + BN to standardowy blok
model_orig = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True)
).to(DEVICE)

# Musimy przełączyć w tryb eval(), żeby BN używał zapisanych statystyk (running_mean/var),
# a nie liczył ich z batcha. Fuzja działa tylko w eval!
model_orig.eval()

print("Model oryginalny:")
print(model_orig)

Model oryginalny:
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
)


## Ręczna Implementacja Fuzji

To jest kod "Black Belt". Wyciągamy wagi z obu warstw i tworzymy nowe wagi metodą algebraiczną.

In [2]:
def fuse_conv_bn(conv, bn):
    """
    Łączy wagi Conv2d i BatchNorm2d w jedną warstwę Conv2d.
    """
    # 1. Pobieramy parametry
    with torch.no_grad():
        # Wagi konwolucji
        w_conv = conv.weight.clone()
        # Bias konwolucji (może być None)
        b_conv = conv.bias.clone() if conv.bias is not None else torch.zeros_like(bn.running_mean)
        
        # Parametry BN
        mean = bn.running_mean
        var_sqrt = torch.sqrt(bn.running_var + bn.eps)
        gamma = bn.weight # scale
        beta = bn.bias    # shift
        
        # 2. Obliczamy nowe wagi (W_new)
        # Musimy dopasować wymiary do mnożenia (C_out, 1, 1, 1)
        scale_factor = gamma / var_sqrt
        w_new = w_conv * scale_factor.view(-1, 1, 1, 1)
        
        # 3. Obliczamy nowy bias (b_new)
        b_new = (b_conv - mean) * scale_factor + beta
        
        # 4. Tworzymy nową warstwę
        fused_conv = nn.Conv2d(
            in_channels=conv.in_channels,
            out_channels=conv.out_channels,
            kernel_size=conv.kernel_size,
            stride=conv.stride,
            padding=conv.padding,
            bias=True # Teraz bias jest obowiązkowy (nawet jak wcześniej go nie było)
        )
        
        # Wgrywamy obliczone parametry
        fused_conv.weight.copy_(w_new)
        fused_conv.bias.copy_(b_new)
        
        return fused_conv

# Zastosujmy to
fused_conv_layer = fuse_conv_bn(model_orig[0], model_orig[1])

# Budujemy nowy model (bez BN!)
model_fused = nn.Sequential(
    fused_conv_layer,
    model_orig[2] # ReLU zostaje
).to(DEVICE)

model_fused.eval()

print("\nModel po fuzji (Zniknął BatchNorm!):")
print(model_fused)


Model po fuzji (Zniknął BatchNorm!):
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
)


## Weryfikacja Numeryczna

Czy matematyka nie kłamie? Sprawdźmy, czy dla tego samego wejścia oba modele dają **identyczny** wynik.

In [3]:
x = torch.randn(1, 3, 224, 224).to(DEVICE)

with torch.no_grad():
    out_orig = model_orig(x)
    out_fused = model_fused(x)

# Sprawdzamy różnicę
diff = (out_orig - out_fused).abs().max().item()

print(f"Maksymalna różnica w wynikach: {diff:.8f}")

if diff < 1e-5:
    print("✅ SUKCES! Modele są matematycznie równoważne.")
else:
    print("❌ COŚ POSZŁO NIE TAK. Różnica jest zbyt duża.")

Maksymalna różnica w wynikach: 0.00000119
✅ SUKCES! Modele są matematycznie równoważne.


## Benchmark: Ile zyskaliśmy?

W małym modelu różnica może być znikoma (narzut Pythona). Ale w dużym ResNet, gdzie takich bloków jest 50, zyskujemy brak 50 operacji odczytu/zapisu pamięci dla BN.

*Uwaga: Na małym tensorze i w Pythonie narzut pętli pomiarowej może ukryć zysk, ale na poziomie CUDA kernel to jest czysty zysk.*

In [4]:
# BENCHMARK
# Musimy zrobić dużo powtórzeń, żeby zobaczyć różnicę (mikrosekundy)
iters = 5000

# 1. Warmup (Rozgrzewka GPU)
for _ in range(100):
    _ = model_orig(x)
    _ = model_fused(x)

torch.cuda.synchronize() if DEVICE == "cuda" else None

# 2. Pomiar Oryginału
start = time.time()
for _ in range(iters):
    _ = model_orig(x)
torch.cuda.synchronize() if DEVICE == "cuda" else None
end = time.time()
time_orig = end - start

# 3. Pomiar Zoptymalizowanego
start = time.time()
for _ in range(iters):
    _ = model_fused(x)
torch.cuda.synchronize() if DEVICE == "cuda" else None
end = time.time()
time_fused = end - start

print(f"Oryginał (Conv+BN): {time_orig:.4f} s")
print(f"Fused (Conv):       {time_fused:.4f} s")
print(f"🚀 Przyspieszenie: {time_orig / time_fused:.2f}x")

Oryginał (Conv+BN): 0.7202 s
Fused (Conv):       0.4779 s
🚀 Przyspieszenie: 1.51x


## 🥋 Black Belt Summary

1.  **Dlaczego warto?**
    *   **Mniej operacji:** Usuwamy BN z grafu obliczeniowego.
    *   **Mniej pamięci:** Nie musimy wczytywać parametrów BN (średnia, wariancja, gamma, beta) z VRAM.
    *   **Mniejszy plik:** Model zajmuje mniej miejsca na dysku.
2.  **Kiedy to robić?**
    *   **Zawsze** przed eksportem do ONNX lub deploymentem na urządzenia mobilne.
    *   W PyTorch Lightning i bibliotece `torchvision` są często gotowe funkcje (np. `torch.nn.utils.fuse_conv_bn_eval`), ale teraz wiesz, jak działają matematycznie.
3.  **Wymóg:** Model musi być w trybie `.eval()`. Fuzja podczas treningu (`.train()`) jest niemożliwa, bo BN musi wtedy dynamicznie aktualizować statystyki.